In [1]:
# Cell 1: Imports and Setup
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import random
import os
from mnist_skeptic_v9 import skeptic_v9

# Set up GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set random seed for reproducibility
random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

Using device: cuda


In [2]:
# Cell 2: Data Loading and Transformation
transform = transforms.Compose([
    transforms.Resize((16, 16), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=15),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: 1 - x if random.random() < 0.5 else x),
    transforms.Lambda(lambda x: x + 0.4 * torch.randn_like(x)),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True)

In [5]:
# Cell 3: Training Function
# Cell 3: Training Function (Updated)
def train_model(model, train_loader, test_loader, epochs=20, patience=3, lr=0.001):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)
        
    best_accuracy = 0
    patience_counter = 0
    
    for epoch in range(epochs):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
        
        accuracy = 100 * correct / total
        print(f'Epoch {epoch+1}/{epochs}, Accuracy: {accuracy:.2f}%')
        
        scheduler.step(100 - accuracy)
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    return model, best_accuracy

In [6]:
# Cell 4: Ensemble Training (Updated)
def train_ensemble(num_models=20):
    ensemble = []
    for i in range(num_models):
        print(f"\nTraining model {i+1}/{num_models}")
        
        # Set a different random seed for each model
        seed = 42 + i
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
        
        model = skeptic_v9()
        
        # Slightly modify learning rate for each model
        lr = 0.001 * (1 + random.uniform(-0.1, 0.1))
        
        trained_model, accuracy = train_model(model, train_loader, test_loader, lr=lr)
        ensemble.append((trained_model, accuracy))
        print(f"Model {i+1} best accuracy: {accuracy:.2f}%")
    return ensemble

ensemble = train_ensemble(num_models=20)


Training model 1/20
Epoch 1/20, Accuracy: 48.42%
Epoch 2/20, Accuracy: 53.37%
Epoch 3/20, Accuracy: 56.78%
Epoch 4/20, Accuracy: 58.64%
Epoch 5/20, Accuracy: 60.14%
Epoch 6/20, Accuracy: 61.50%
Epoch 7/20, Accuracy: 62.06%
Epoch 8/20, Accuracy: 62.72%
Epoch 9/20, Accuracy: 63.40%
Epoch 10/20, Accuracy: 64.16%
Epoch 11/20, Accuracy: 64.15%
Epoch 12/20, Accuracy: 64.26%
Epoch 13/20, Accuracy: 64.64%
Epoch 14/20, Accuracy: 64.72%
Epoch 15/20, Accuracy: 64.51%
Epoch 16/20, Accuracy: 64.36%
Epoch 17/20, Accuracy: 65.00%
Epoch 18/20, Accuracy: 64.93%
Epoch 19/20, Accuracy: 65.59%
Epoch 20/20, Accuracy: 66.54%
Model 1 best accuracy: 66.54%

Training model 2/20
Epoch 1/20, Accuracy: 43.96%
Epoch 2/20, Accuracy: 50.46%
Epoch 3/20, Accuracy: 52.70%
Epoch 4/20, Accuracy: 55.57%
Epoch 5/20, Accuracy: 58.72%
Epoch 6/20, Accuracy: 60.06%
Epoch 7/20, Accuracy: 60.18%
Epoch 8/20, Accuracy: 62.01%
Epoch 9/20, Accuracy: 62.93%
Epoch 10/20, Accuracy: 62.87%
Epoch 11/20, Accuracy: 64.29%
Epoch 12/20, Acc

In [7]:
# Cell 5: Model Saving
def save_ensemble(ensemble, directory='best_boi_models'):
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    for i, (model, accuracy) in enumerate(ensemble):
        model_path = os.path.join(directory, f'model_{i+1}_acc_{accuracy:.2f}.pth')
        torch.save(model.state_dict(), model_path)
        print(f"Model {i+1} saved to {model_path}")

save_ensemble(ensemble)

Model 1 saved to best_boi_models\model_1_acc_66.54.pth
Model 2 saved to best_boi_models\model_2_acc_66.34.pth
Model 3 saved to best_boi_models\model_3_acc_67.64.pth
Model 4 saved to best_boi_models\model_4_acc_66.18.pth
Model 5 saved to best_boi_models\model_5_acc_66.35.pth
Model 6 saved to best_boi_models\model_6_acc_67.36.pth
Model 7 saved to best_boi_models\model_7_acc_68.04.pth
Model 8 saved to best_boi_models\model_8_acc_66.93.pth
Model 9 saved to best_boi_models\model_9_acc_67.54.pth
Model 10 saved to best_boi_models\model_10_acc_67.35.pth
Model 11 saved to best_boi_models\model_11_acc_66.82.pth
Model 12 saved to best_boi_models\model_12_acc_67.03.pth
Model 13 saved to best_boi_models\model_13_acc_66.77.pth
Model 14 saved to best_boi_models\model_14_acc_66.48.pth
Model 15 saved to best_boi_models\model_15_acc_67.12.pth
Model 16 saved to best_boi_models\model_16_acc_67.02.pth
Model 17 saved to best_boi_models\model_17_acc_66.84.pth
Model 18 saved to best_boi_models\model_18_acc_67